# 1 Introduction and REST API

Objectives
 * Verify HDF Lab is working correctly
 * Learn how to make REST requests to the HSDS Server

In [ ]:
# the server endpoint 
hs_endpoint = "http://hsdshdflab.hdfgroup.org"   

In [ ]:
# request is a popular python library for making http requests
import requests
import json

In [ ]:
rsp = requests.get(hs_endpoint + "/about")

In [6]:
rsp.status_code  # should be 200

200

In [7]:
rsp.json()

{'state': 'READY',
 'name': 'hsdstest',
 'greeting': 'Welcome to HSDS!',
 'hsds_version': '0.1',
 'start_time': 1531692925,
 'about': 'HSDS is a webservice for HDF data'}

Problem: "/info" is another request supported by the server.  What does it return?

In [8]:
# get information about a HDF5 "file" on the server
params={"domain":"/shared/tall.h5"}
rsp = requests.get(hs_endpoint + "/", params=params)
rsp.status_code

200

In [9]:
rsp_json = rsp.json()
rsp_json

{'created': 1531174596.0696847,
 'hrefs': [{'rel': 'self',
   'href': 'hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5'},
  {'rel': 'database',
   'href': 'hsdshdflab.hdfgroup.org/datasets?domain=/shared/tall.h5'},
  {'rel': 'groupbase',
   'href': 'hsdshdflab.hdfgroup.org/groups?domain=/shared/tall.h5'},
  {'rel': 'typebase',
   'href': 'hsdshdflab.hdfgroup.org/datatypes?domain=/shared/tall.h5'},
  {'rel': 'root',
   'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016?domain=/shared/tall.h5'},
  {'rel': 'acls',
   'href': 'hsdshdflab.hdfgroup.org/acls?domain=/shared/tall.h5'},
  {'rel': 'parent', 'href': 'hsdshdflab.hdfgroup.org/?domain=/shared'}],
 'lastModified': 1531174596.0696847,
 'owner': 'admin',
 'class': 'domain',
 'root': 'g-be5996fa-83c5-11e8-a8e6-0242ac120016'}

In [10]:
root_id = rsp_json["root"]
root_id

'g-be5996fa-83c5-11e8-a8e6-0242ac120016'

In [11]:
# getinfo about the root group
rsp = requests.get(hs_endpoint + "/groups/" + root_id, params=params)
rsp.status_code

200

In [12]:
root_json = rsp.json()
root_json

{'created': 1531174596.0384045,
 'lastModified': 1531174597.2827835,
 'hrefs': [{'rel': 'self',
   'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016?domain=/shared/tall.h5'},
  {'rel': 'links',
   'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016/links?domain=/shared/tall.h5'},
  {'rel': 'root',
   'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016?domain=/shared/tall.h5'},
  {'rel': 'home', 'href': 'hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5'},
  {'rel': 'attributes',
   'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016/attributes?domain=/shared/tall.h5'}],
 'linkCount': 2,
 'id': 'g-be5996fa-83c5-11e8-a8e6-0242ac120016',
 'attributeCount': 2,
 'domain': '/shared/tall.h5',
 'root': 'g-be5996fa-83c5-11e8-a8e6-0242ac120016'}

Problem: replace the root_id in the last request with "g-1234".  What status code do you get?

In [13]:
# get the dataset at /g1/g1.1/dset1.1.1
params["h5path"] = "/g1/g1.1/dset1.1.1"
rsp = requests.get(hs_endpoint + "/datasets/", params=params)
rsp.status_code

200

In [14]:
dset_json = rsp.json()
dset_json

{'layout': {'class': 'H5D_CHUNKED', 'dims': [10, 10]},
 'lastModified': 1531174596,
 'created': 1531174596,
 'hrefs': [{'href': 'hsdshdflab.hdfgroup.org/datasets/d-be8bace4-83c5-11e8-90e7-0242ac120013?domain=/shared/tall.h5',
   'rel': 'self'},
  {'href': 'hsdshdflab.hdfgroup.org/groups/g-be5996fa-83c5-11e8-a8e6-0242ac120016?domain=/shared/tall.h5',
   'rel': 'root'},
  {'href': 'hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5', 'rel': 'home'},
  {'href': 'hsdshdflab.hdfgroup.org/datasets/d-be8bace4-83c5-11e8-90e7-0242ac120013/attributes?domain=/shared/tall.h5',
   'rel': 'attributes'},
  {'href': 'hsdshdflab.hdfgroup.org/datasets/d-be8bace4-83c5-11e8-90e7-0242ac120013/value?domain=/shared/tall.h5',
   'rel': 'data'}],
 'domain': '/shared/tall.h5',
 'shape': {'maxdims': [10, 10], 'class': 'H5S_SIMPLE', 'dims': [10, 10]},
 'id': 'd-be8bace4-83c5-11e8-90e7-0242ac120013',
 'root': 'g-be5996fa-83c5-11e8-a8e6-0242ac120016',
 'attributeCount': 2,
 'type': {'base': 'H5T_STD_I32BE', 'class': '

In [15]:
# read all the values from the dataset
dset_id = dset_json["id"]
del params["h5path"]  # don't need this now
rsp = requests.get(hs_endpoint + "/datasets/" + dset_id + "/value", params=params)
rsp.status_code

200

In [16]:
value_json = rsp.json()
value_json["value"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [0, 3, 6, 9, 12, 15, 18, 21, 24, 27],
 [0, 4, 8, 12, 16, 20, 24, 28, 32, 36],
 [0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
 [0, 6, 12, 18, 24, 30, 36, 42, 48, 54],
 [0, 7, 14, 21, 28, 35, 42, 49, 56, 63],
 [0, 8, 16, 24, 32, 40, 48, 56, 64, 72],
 [0, 9, 18, 27, 36, 45, 54, 63, 72, 81]]

In [17]:
# get just the upper 3x4 elements
params["select"] = "[0:3, 0:4]"
rsp = requests.get(hs_endpoint + "/datasets/" + dset_id + "/value", params=params)
rsp.status_code

200

In [18]:
subarray_json = rsp.json()
subarray_json["value"]

[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 4, 6]]

In [19]:
# read elements (i,i) for i in range 0-9
points = []
for i in range(10):
    points.append([i,i])
body = {"points": points}
del params["select"] # remove select so we will read across entire dataset
rsp = requests.post(hs_endpoint + "/datasets/" + dset_id + "/value", params=params, data=json.dumps(body))
rsp.status_code

200

In [20]:
points_json = rsp.json()
points_json["value"]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [21]:
# we can also get data values as a binary stream
headers = {"accept": "application/octet-stream"}
rsp = requests.post(hs_endpoint + "/datasets/" + dset_id + "/value", params=params, data=json.dumps(body), headers=headers)
rsp.status_code


200

In [22]:
rsp.content

b'\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\t\x00\x00\x00\x10\x00\x00\x00\x19\x00\x00\x00$\x00\x00\x001\x00\x00\x00@\x00\x00\x00Q'